# Support Vector Regression Process and Analysis for Solar Data

## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_validate, KFold
from sklearn import svm
from sklearn.preprocessing import StandardScaler

## Data Preprocessing

First, we read in the dataset.

In [2]:
# Run these commands if running notebooks on Jupyter, or locally.

df = pd.read_csv("../data/solar.csv")
df.head(5)

In [3]:
# Run these commands when running the notebooks on Google Colab.

df = pd.read_csv('https://raw.githubusercontent.com/AidanNeeson/renewable-ml/main/data/solar.csv')
df.head(5)

Now, we must shuffle the datasets to reduce bias.

In [3]:
df = df.sample(frac=1)
df.head(5)

Looking at each dataset, we can identify which variables we want to use
for our models.

In [5]:
X = df.loc[:, ['lat','long','capacity']]
y_energy = df['generated_energy'].values
y_cost = df['cost'].values

Now we split into training and testing sets, reserving about 80% for
training and 20% for testing.

In [6]:
X_train = X[:9500]
X_test = X[9500:]
y_energy_train = y_energy[:9500]
y_energy_test = y_energy[9500:]
y_cost_train = y_cost[:9500]
y_cost_test = y_cost[9500:]

Models typically perform better when input values are within a certain
range, like \[-1, 1\] for example. We scale the data points
appropriately.

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

X_train

array([[ 2.03229033,  0.92873139, -0.43905719],
       [-0.5962383 , -1.96682005, -0.43098755],
       [-0.02700756, -0.16672944, -0.43098755],
       ...,
       [-0.12947216, -0.24196866, -0.43905719],
       [ 2.41359832,  0.38433555, -0.43904104],
       [-0.0831422 ,  0.65286873, -0.43098755]])

## Training the Models

Now that the data is pre-processed accordingly, the models can be
trained and fit.

In [8]:
energy_reg = svm.SVR()
cost_reg = svm.SVR()
energy_reg.fit(X_train, y_energy_train)
cost_reg.fit(X_train, y_cost_train)

SVR() In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.   SVR ? Documentation for SVR i Fitted SVR()

With a trained model, predictions can now be made.

In [10]:
energy_display = y_energy_test
cost_display = y_cost_test
energy_preds = energy_reg.predict(X_test)
cost_preds = cost_reg.predict(X_test)
print("Predictions")
print("----------------------")
for i in range(3):
    print(f"predicted energy: {energy_preds[i]:.2f}\tactual energy: {energy_display[i]:.2f}\tpredicted cost: {cost_preds[i]:.2f}\tactual cost: {cost_display[i]:.2f}")

Predictions
----------------------
predicted energy: 2494.27   actual energy: 6.13 predicted cost: 1913464.63  actual cost: 4905.60
predicted energy: 2438.26   actual energy: 2452.80  predicted cost: 1913207.65  actual cost: 1913184.00
predicted energy: 2753.60   actual energy: 183.96   predicted cost: 1913890.42  actual cost: 172922.40

## Testing and Analyzing the Models

This section contains metrics gathering and other figures that visualize
the models and its results.

### Metrics

#### Scores and Error Values

The score being recored are the R2 score, Root Mean Squared Error
(RMSE), and Mean Absolute Percentage Error (MAPE).

In [11]:
energy_r2 = metrics.r2_score(y_energy_test, energy_preds, multioutput="raw_values")
energy_rmse = metrics.root_mean_squared_error(y_energy_test, energy_preds, multioutput="raw_values")
energy_mape = metrics.mean_absolute_percentage_error(y_energy_test, energy_preds, multioutput="raw_values")

cost_r2 = metrics.r2_score(y_cost_test, cost_preds, multioutput="raw_values")
cost_rmse = metrics.root_mean_squared_error(y_cost_test, cost_preds, multioutput="raw_values")
cost_mape = metrics.mean_absolute_percentage_error(y_cost_test, cost_preds, multioutput="raw_values")

print("Metric\tScore")
print("-----------------------")
print(f"energy_r2\t{energy_r2}\ncost_r2\t{cost_r2}\nenergy_rmse\t{energy_rmse}\ncost_rmse\t{cost_rmse}\nenergy_mape\t{energy_mape}\ncost_mape\t{cost_mape}")

Metric  Score
-----------------------
energy_r2   [-0.21012622]
cost_r2 [-0.20850399]
energy_rmse [880927.68663408]
cost_rmse   [6.87646055e+08]
energy_mape [81.95543371]
cost_mape   [81.30935189]

#### K-Fold Cross Validation

This cross validation splits up the dataset into 10 unique folds, which
are then used to test a model. The model is then scored using the same
metrics outlined above: R2, RMSE, MAPE. This ensures the scoring is
rigorous, and the *entire* dataset is used.

In [12]:
kf = KFold(n_splits=10, random_state=0, shuffle=True)
kf_cv_scores = cross_validate(energy_reg, X, y_energy, cv=kf, scoring={"r2":metrics.make_scorer(score_func=metrics.r2_score),
 "rmse":metrics.make_scorer(score_func=metrics.root_mean_squared_error),
 "mape":metrics.make_scorer(score_func=metrics.mean_absolute_percentage_error)})
kf_cv_df = pd.DataFrame.from_dict(kf_cv_scores)
means = kf_cv_df.mean()
print("10-Fold Cross Validation Scores")
print("----------------------------------------------------")
print(f"R2 Average: {means.iloc[2]}")
print(f"RMSE Average: {means.iloc[3]}")
print(f"MAPE Average: {means.iloc[4]}")
kf_cv_df

10-Fold Cross Validation Scores
----------------------------------------------------
R2 Average: -0.19080402178052483
RMSE Average: 835719.6572480934
MAPE Average: 83.11834411696152

In [13]:
kf = KFold(n_splits=10, random_state=0, shuffle=True)
kf_cv_scores = cross_validate(cost_reg, X, y_cost, cv=kf, scoring={"r2":metrics.make_scorer(score_func=metrics.r2_score),
 "rmse":metrics.make_scorer(score_func=metrics.root_mean_squared_error),
 "mape":metrics.make_scorer(score_func=metrics.mean_absolute_percentage_error)})
kf_cv_df = pd.DataFrame.from_dict(kf_cv_scores)
means = kf_cv_df.mean()
print("10-Fold Cross Validation Scores")
print("----------------------------------------------------")
print(f"R2 Average: {means.iloc[2]}")
print(f"RMSE Average: {means.iloc[3]}")
print(f"MAPE Average: {means.iloc[4]}")
kf_cv_df

10-Fold Cross Validation Scores
----------------------------------------------------
R2 Average: -0.19100059475135955
RMSE Average: 657874231.9115219
MAPE Average: 83.9877821694812

### Graphs

Graphs of the SVR model fits on each on the input features, for each
target.

In [18]:
plot_lat_x = X[9500:].loc[:,['lat']].sort_values(by=['lat'])
plot_long_x = X[9500:].loc[:,['long']].sort_values(by=['long'])
plot_cap_x = X[9500:].loc[:,['capacity']].sort_values(by=['capacity'])
plot_energy_y = pd.DataFrame(energy_preds).loc[:,[0]].sort_values(by=[0])

figure, axis = plt.subplots(3)

figure.set_size_inches(15,15)

axis[0].scatter(X.loc[:,["lat"]], y_energy, color='blue', label='Data', s=5)
axis[0].plot(plot_lat_x, plot_energy_y, color='red',lw=2, label="Generated Energy Model")
axis[0].set_xlabel("Latitude")
axis[0].set_ylabel("Generated Energy(MWh)")
axis[0].set_title("Support Vector Regression: Generated Energy vs. Latitude")
axis[0].legend()

axis[1].scatter(X.loc[:,["long"]], y_energy, color='blue', label='Data', s=5)
axis[1].plot(plot_long_x, plot_energy_y, color='red',lw=2, label="Generated Energy Model")
axis[1].set_xlabel("Longitude")
axis[1].set_ylabel("Generated Energy(MWh)")
axis[1].set_title("Support Vector Regression: Generated Energy vs. Longitude")
axis[1].legend()

axis[2].scatter(X.loc[:,["capacity"]], y_energy, color='blue', label='Data', s=5)
axis[2].plot(plot_cap_x, plot_energy_y, color='red',lw=2, label="Generated Energy Model")
axis[2].set_xlabel("Capacity(MW)")
axis[2].set_ylabel("Generated Energy(MWh)")
axis[2].set_title("Support Vector Regression: Generated Energy vs. Capacity")
axis[2].legend()

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=.4, 
                    hspace=.4)

plt.show()

In [19]:
plot_cost_y = pd.DataFrame(cost_preds).loc[:,[0]].sort_values(by=[0])

figure, axis = plt.subplots(3)

figure.set_size_inches(15,15)

axis[0].scatter(X.loc[:,["lat"]], y_cost, color='blue', label='Data', s=5)
axis[0].plot(plot_lat_x, plot_cost_y, color='red',lw=2, label="Cost Model")
axis[0].set_xlabel("Latitude")
axis[0].set_ylabel("Cost($)")
axis[0].set_title("Support Vector Regression: Cost vs. Latitude")
axis[0].legend()

axis[1].scatter(X.loc[:,["long"]], y_cost, color='blue', label='Data', s=5)
axis[1].plot(plot_long_x, plot_cost_y, color='red',lw=2, label="Cost Model")
axis[1].set_xlabel("Longitude")
axis[1].set_ylabel("Cost($)")
axis[1].set_title("Support Vector Regression: Cost vs. Longitude")
axis[1].legend()

axis[2].scatter(X.loc[:,["capacity"]], y_cost, color='blue', label='Data', s=5)
axis[2].plot(plot_cap_x, plot_cost_y, color='red',lw=2, label="Cost Model")
axis[2].set_xlabel("Capacity(MW)")
axis[2].set_ylabel("Cost($)")
axis[2].set_title("Support Vector Regression: Cost vs. Capacity")
axis[2].legend()

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    wspace=.4, 
                    hspace=.4)

plt.show()